# Regression

This example will guide through a very noisy regression problem. First we have to create the regression data. The data will originate from
the function $x \cdot sin(x)$.

In [ ]:
import pandas as pd
import numpy as np

def f(x):
    return x * np.sin(x)

def generate_data(func, x_lower, x_upper, n):
    '''
    Generate a ``DataFrame`` of ``n`` data samples with additive noise from the function ``func``.
    '''
    X = np.atleast_2d(np.random.uniform(-20, 0.0, size=int(n / 2))).T
    X = np.vstack((np.atleast_2d(np.random.uniform(0, 10.0, size=int(n / 2))).T, X))
    X = X.astype(np.float32)
    X = np.array(list(sorted(X)))

    # Observations
    y = func(X).ravel()

    # Add some noise
    dy = 1.5 + .5 * np.random.random(y.shape)
    y += np.random.normal(0, dy)
    y = y.astype(np.float32)

    return pd.DataFrame(data={'x': X.ravel(), 'y': y})

df = generate_data(f, -20, 10, 1000)
df

Lets have a look at the data visually:

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

plt.plot(df['x'].values, df['y'].values, '.', markersize=5, label='Training data')
plt.show()

We can clearly see the original function, and we can observe that there is more variance towards higher x. In general, we don't want a model that assumes independent noise, and we don't want to specify much prior knowledge since it is only available rarely. Let's specify a JPT that focuses on predicting y accurately.

In [ ]:
from jpt.variables import NumericVariable, VariableMap
from jpt.trees import JPT

# Construct the variables, soften up x via blur
varx = NumericVariable('x', blur=0.05)
vary = NumericVariable('y')

# enable a focus on the parameters for y with the targets keyword
jpt = JPT(variables=[varx, vary], targets=[vary], min_samples_leaf=.01)

jpt.learn(data=df)

Since the data is 2D we can get a visual representation of the predictions of the JPT. Let's plot the predictions and get an idea of the distribution. We will plot the expectation of y for every datapoint on the x-axis and confidence intervals that explain 95% of the variance.

In [ ]:
# Mesh the input space for evaluations of the real function,
# the prediction and its MSE
xx = np.atleast_2d(np.linspace(-20, 15, 500)).astype(np.float32).T

# Apply the JPT model
confidence = .95

my_predictions = [jpt.expectation([vary],
                                  evidence={varx: x_},
                                  confidence_level=confidence,
                                  fail_on_unsatisfiability=False) for x_ in xx.ravel()]
y_pred_ = [(p[vary].result if p is not None else None) for p in my_predictions]
y_lower_ = [(p[vary].lower if p is not None else None) for p in my_predictions]
y_upper_ = [(p[vary].upper if p is not None else None) for p in my_predictions]

# posterior = jpt.posterior([varx], {vary: 0})

# Plot the function, the prediction and the 90% confidence interval based on the MSE
plt.plot(xx, f(xx), color='black', linestyle=':', linewidth='2', label=r'$f(x) = x\,\sin(x)$')
plt.plot(df['x'].values, df['y'].values, '.', color='gray', markersize=5, label='Training data')
plt.plot(xx, y_pred_, 'm-', label='JPT Prediction', linewidth=2)
plt.plot(xx, y_lower_, 'y--', label='%.1f%% Confidence bands' % (confidence * 100))
plt.plot(xx, y_upper_, 'y--')
plt.plot(xx, np.array([jpt.pdf(VariableMap([(varx, x_), (vary, 0)])) for x_ in xx.ravel().astype(np.float64)]),
         label='Posterior')

plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.ylim(-10, 20)
plt.xlim(-25, 15)
plt.legend(loc='upper left')
plt.title(r'2D Regression Example ($\vartheta=%.2f\%%$)' % (confidence * 100))
plt.grid()
plt.rcParams['figure.figsize'] = [40, 20]
plt.show()

As we can see the expectations and variances predicted by the JPT characterize this dataset well and the variance even adjusts to the different x values. Impressive!